<a href="https://colab.research.google.com/github/ysut/weslink/blob/main/weslink_beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use
Please click ▶ buttons.  

# Step 1.  
Let's upload your files.  
Excel file (.xlsx) is only available now (12/20/2022).

In [ ]:
!pip install liftover
import pathlib
import openpyxl
from liftover import get_lifter
from google.colab import files

uploaded = files.upload()

# Step 2.  
Get a path to your file from 📁. You can copy a path by right click.   
Then, please paste in the next form.

In [ ]:
input_file = input('Please enter a path: ')

# Step 3.
まとめてRUNしてください．

# RUN

In [23]:
hgmd_link = '1'
ucsc_link = '2'
franklin_link = '3'
decipher_link = '4'
symbol_column = 'Gene.refGene'

# Choose one page from 'assessment-tools', 'acmg-classification', 'publications', and 'gene-assessment'
franklin_page = 'assessment-tools' 

is_hgmd = True
is_ucsc = True
is_franklin = True
is_decipher = True

# Display the range specified plus or minus bp
ucsc_width: int = 30 

## Code  
There is main code. You can modify it as needed.

In [24]:
class HyperLinks:
    def __init__(self, input_file, hgmd_link, ucsc_link, franklin_link, decipher_link, symbol_column):
        self.input_file = input_file
        self.workbook = openpyxl.load_workbook(self.input_file)
        self.sheet_names = self.workbook.sheetnames
        self.sheets = self.workbook.worksheets
        self.symbol_column = symbol_column
        self.insert_column_hgmd = openpyxl.utils.cell.get_column_letter(int(hgmd_link))
        self.insert_column_ucsc = openpyxl.utils.cell.get_column_letter(int(ucsc_link))
        self.insert_column_frnk = openpyxl.utils.cell.get_column_letter(int(franklin_link)) 
        self.insert_column_dcpr = openpyxl.utils.cell.get_column_letter(int(decipher_link))
        self.input_path = pathlib.Path(self.input_file)
        self.input_file_name = self.input_path.stem
    
    def __del__(self):
        self.workbook.close()
        
    def _search_column_letter(self, keywords):
        result = {}
        for i, sheet in enumerate(self.sheets):
            key_letter_dict = {}
            for cell in sheet['1']:
                try:
                    value = str(cell.value)
                except: 
                    continue
                for keyword in keywords:
                    if value == keyword:
                        letter = cell.column_letter
                        key_letter_dict[keyword] = letter
                        break
                    else:
                        pass
            result[self.sheet_names[i]] = key_letter_dict
        return result 
    
    def hyperlink_hgmd(self):
        column_letters = self._search_column_letter([self.symbol_column])
        for sheet_num, sheet in enumerate(self.sheets):
            symbol_column_letter = column_letters[self.sheet_names[sheet_num]][self.symbol_column]
            max_row = sheet.max_row
            for index in range(max_row):
                gene_symbol = sheet[f'{symbol_column_letter}{index + 1}'].value
                hgmd_url = f'https://my.qiagendigitalinsights.com/bbp/view/hgmd/pro/gene.php?gene={gene_symbol}'
                sheet[f'{self.insert_column_hgmd}{index + 1}'].hyperlink = hgmd_url
    
    def hyperlink_ucsc(self):
        column_letters = self._search_column_letter([self.symbol_column, 'CHROM', 'Start', 'End'])
        for sheet_num, sheet in enumerate(self.sheets):
            symbol_column_letter = column_letters[self.sheet_names[sheet_num]][self.symbol_column]
            chrom_column_letter = column_letters[self.sheet_names[sheet_num]]['CHROM']
            start_column_letter = column_letters[self.sheet_names[sheet_num]]['Start']
            end_column_letter = column_letters[self.sheet_names[sheet_num]]['End']
            max_row = sheet.max_row
            for index in range(max_row):
                if index != 0:
                    gene_symbol = sheet[f'{symbol_column_letter}{index + 1}'].value
                    chrom = sheet[f'{chrom_column_letter}{index + 1}'].value
                    start = int(sheet[f'{start_column_letter}{index + 1}'].value)
                    end = int(sheet[f'{end_column_letter}{index + 1}'].value)
                    ucsc_url = f'https://genome-asia.ucsc.edu/cgi-bin/hgTracks?db=hg19&highlight=hg19.chr{chrom}%3A{start}-{end}&position=chr{chrom}%3A{start - ucsc_width}-{end + ucsc_width}'
                    sheet[f'{self.insert_column_ucsc}{index + 1}'].hyperlink = ucsc_url
                else:
                    pass

    def hyperlink_franklin(self):
        column_letters = self._search_column_letter([self.symbol_column, 'CHROM', 'POS', 'REF', 'ALT'])
        for sheet_num, sheet in enumerate(self.sheets):
            symbol_column_letter = column_letters[self.sheet_names[sheet_num]][self.symbol_column]
            chrom_column_letter = column_letters[self.sheet_names[sheet_num]]['CHROM']
            pos_column_letter = column_letters[self.sheet_names[sheet_num]]['POS']
            ref_column_letter = column_letters[self.sheet_names[sheet_num]]['REF']
            alt_column_letter = column_letters[self.sheet_names[sheet_num]]['ALT']
            max_row = sheet.max_row
            for index in range(max_row):
                if index != 0:
                    gene_symbol = sheet[f'{symbol_column_letter}{index + 1}'].value
                    chrom = sheet[f'{chrom_column_letter}{index + 1}'].value
                    pos = int(sheet[f'{pos_column_letter}{index + 1}'].value)
                    ref = sheet[f'{ref_column_letter}{index + 1}'].value
                    alt = sheet[f'{alt_column_letter}{index + 1}'].value
                    frnk_url = f'https://franklin.genoox.com/clinical-db/variant/snp/chr{chrom}-{pos}-{ref}-{alt}?app={franklin_page}'
                    sheet[f'{self.insert_column_frnk}{index + 1}'].hyperlink = frnk_url
                else:
                    pass

    def hyperlink_decipher(self):
        column_letters = self._search_column_letter([self.symbol_column, 'CHROM', 'POS', 'REF', 'ALT'])
        for sheet_num, sheet in enumerate(self.sheets):
            symbol_column_letter = column_letters[self.sheet_names[sheet_num]][self.symbol_column]
            chrom_column_letter = column_letters[self.sheet_names[sheet_num]]['CHROM']
            pos_column_letter = column_letters[self.sheet_names[sheet_num]]['POS']
            ref_column_letter = column_letters[self.sheet_names[sheet_num]]['REF']
            alt_column_letter = column_letters[self.sheet_names[sheet_num]]['ALT']
            max_row = sheet.max_row
            for index in range(max_row):
                if index != 0:
                    gene_symbol = sheet[f'{symbol_column_letter}{index + 1}'].value
                    chrom = sheet[f'{chrom_column_letter}{index + 1}'].value
                    pos = int(sheet[f'{pos_column_letter}{index + 1}'].value)
                    ref = sheet[f'{ref_column_letter}{index + 1}'].value
                    alt = sheet[f'{alt_column_letter}{index + 1}'].value
                    converter = get_lifter('hg19', 'hg38')
                    try: 
                        pos_hg38 = converter[chrom][pos][0][1]
                    except IndexError:
                        dcphr_url = f'https://www.deciphergenomics.org/gene/{gene_symbol}/overview/protein-genomic-info'
                        sheet[f'{self.insert_column_dcpr}{index + 1}'].hyperlink = dcpr_url
                        continue
                    dcpr_url = f'https://www.deciphergenomics.org/sequence-variant/{chrom}-{pos_hg38}-{ref}-{alt}/genes/{gene_symbol}/protein-genomic-info'
                    sheet[f'{self.insert_column_dcpr}{index + 1}'].hyperlink = dcpr_url
                else:
                    pass 

    def save_dl(self):
        output_name = f'{self.input_file_name}.weslinked.xlsx'
        self.workbook.save(output_name)
        files.download(output_name)


def main():
    linkanno = HyperLinks(input_file, hgmd_link, ucsc_link, franklin_link, decipher_link, symbol_column)

    if is_hgmd:
        linkanno.hyperlink_hgmd()
    else:
        pass

    if is_ucsc:
        linkanno.hyperlink_ucsc()
    else:
        pass
    
    if is_franklin:
        linkanno.hyperlink_franklin()
    else:
        pass
    
    if is_decipher:
        linkanno.hyperlink_decipher()
    else:
        pass

    linkanno.save_dl()
    
    del linkanno

In [ ]:
main()

# Step 4.  
You can delete uploaded files.

In [26]:
!find ./ -type f -name "*.xlsx" | xargs rm -rf